In [1]:
import torch
import torch.nn as nn
from torch.utils.data import DataLoader
from torchvision import transforms
from torchvision import datasets
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
# 展示高清图
from matplotlib_inline import backend_inline
backend_inline.set_matplotlib_formats('svg')

In [3]:
# 制作数据集
# 数据集转换参数
transform = transforms.Compose([
    transforms.ToTensor(), 
    transforms.Normalize(0.1307, 0.3081)
])
# 下载训练集与测试集
train_Data = datasets.MNIST(
    root = 'D:/Jupyter/dataset/mnist/', # 下载路径
    train = True, # 是 train 集
    download = True, # 如果该路径没有该数据集，就下载
    transform = transform # 数据集转换参数
)
test_Data = datasets.MNIST(
    root = 'D:/Jupyter/dataset/mnist/', # 下载路径
    train = False, # 是 test 集
    download = True, # 如果该路径没有该数据集，就下载
    transform = transform # 数据集转换参数
)

In [4]:
# 批次加载器
train_loader = DataLoader(dataset=train_Data, shuffle=True, batch_size=64)
test_loader = DataLoader(dataset=test_Data, shuffle=False, batch_size=32)

In [5]:
# 搭建神经网络
class DNN(nn.Module):
    def __init__(self):
        ''' 搭建神经网络各层 '''
        super(DNN, self).__init__()
        self.net = nn.Sequential(   # 按顺序搭建各层
            nn.Flatten(),                    # 把图像铺平成一维
            nn.Linear(784, 512), nn.ReLU(),  # 第 1 层：全连接层
            nn.Linear(512, 256), nn.ReLU(),  # 第 2 层：全连接层
            nn.Linear(256, 128), nn.ReLU(),  # 第 3 层：全连接层
            nn.Linear(128, 64), nn.ReLU(),   # 第 4 层：全连接层
            nn.Linear(64, 10)                # 第 5 层：全连接层
        )

    def forward(self, x):
        ''' 前向传播 '''
        y = self.net(x)  # x 即输入数据, 这里的net和__init__()中的net要一致，自己起名
        return y         # y 即输出数据

In [6]:
model = DNN().to('cuda:0') # 创建子类的实例，并搬到 GPU 上
model # 查看该实例的各层

DNN(
  (net): Sequential(
    (0): Flatten(start_dim=1, end_dim=-1)
    (1): Linear(in_features=784, out_features=512, bias=True)
    (2): ReLU()
    (3): Linear(in_features=512, out_features=256, bias=True)
    (4): ReLU()
    (5): Linear(in_features=256, out_features=128, bias=True)
    (6): ReLU()
    (7): Linear(in_features=128, out_features=64, bias=True)
    (8): ReLU()
    (9): Linear(in_features=64, out_features=10, bias=True)
  )
)

In [7]:
# 损失函数的选择
loss_fn = nn.CrossEntropyLoss() # 自带 softmax 激活函数

In [8]:
# 优化算法的选择
learning_rate = 0.01 # 设置学习率
optimizer = torch.optim.SGD(
    model.parameters(),
    lr = learning_rate,
    momentum = 0.5   
)

In [ ]:
# 训练网络
epochs = 5
losses = [] # 记录损失函数变化的列表

for epoch in range(epochs):
    for (x, y) in train_loader: # 获取小批次的 x 与 y
        x, y = x.to('cuda:0'), y.to('cuda:0')
        Pred = model(x) # 一次前向传播（小批量）
        loss = loss_fn(Pred, y) # 计算损失函数
        losses.append(loss.item()) # 记录损失函数的变化
        optimizer.zero_grad() # 清理上一轮滞留的梯度
        loss.backward() # 一次反向传播
        optimizer.step() # 优化内部参数
        
Fig = plt.figure()
plt.plot(range(len(losses)), losses)
plt.show()

In [10]:
# 测试网络
correct = 0
total = 0
with torch.no_grad(): # 该局部关闭梯度计算功能
    for (x, y) in test_loader: # 获取小批次的 x 与 y
        x, y = x.to('cuda:0'), y.to('cuda:0')
        Pred = model(x) # 一次前向传播（小批量）
        _, predicted = torch.max(Pred.data, dim=1)
        correct += torch.sum( (predicted == y) )
        total += y.size(0) 
print(f'测试集精准度: {100*correct/total} %')

测试集精准度: 73.45999908447266 %


In [12]:
# 测试网络
correct = 0
total = 0
with torch.no_grad(): # 该局部关闭梯度计算功能
    for (x, y) in test_loader: # 获取小批次的 x 与 y
        x, y = x.to('cuda:0'), y.to('cuda:0')   
        Pred = model(x) # 一次前向传播（小批量）
        print(Pred.type(), y.type())  
        print(Pred.shape, y.shape)       
        _, predicted = torch.max(Pred.data, dim=1)
        print(predicted.type(), y.type())  
        print(predicted.shape, y.shape)      
        print((predicted == y).all(1).type())   
        print((predicted == y).all(1).shape)    
        correct += torch.sum( (predicted == y) )
        total += y.size(0) 
print(f'测试集精准度: {100*correct/total} %')

torch.cuda.FloatTensor torch.cuda.LongTensor
torch.Size([32, 10]) torch.Size([32])
torch.cuda.LongTensor torch.cuda.LongTensor
torch.Size([32]) torch.Size([32])


IndexError: Dimension out of range (expected to be in range of [-1, 0], but got 1)

In [20]:
# 测试网络
correct = 0
total = 0
with torch.no_grad(): # 该局部关闭梯度计算功能
    for (x, y) in test_loader: # 获取小批次的 x 与 y
        x, y = x.to('cuda:0'), y.to('cuda:0')
        Pred = model(x) # 一次前向传播（小批量）  
        _, predicted = torch.max(Pred.data, dim=1)
        print(Pred.shape, predicted.shape, y.shape) 
        print(Pred[0]) 
        print(predicted[0])  
        print(y[0])  
        print("\n" "\n" "\n")          
        correct += torch.sum( (predicted == y) )
        total += y.size(0) 
print(f'测试集精准度: {100*correct/total} %')

torch.Size([32, 10]) torch.Size([32]) torch.Size([32])
tensor([ -1.0322,   0.0128,   3.2546,   3.8886,  -6.5023,  -2.8038, -10.8745,
         10.9349,  -2.4017,   2.5305], device='cuda:0')
tensor(7, device='cuda:0')
tensor(7, device='cuda:0')




torch.Size([32, 10]) torch.Size([32]) torch.Size([32])
tensor([ -5.5335,   0.4379,  -2.1363,  13.2028,  -7.7333,   7.3093, -10.8330,
         -2.4855,   4.2757,   4.0869], device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')




torch.Size([32, 10]) torch.Size([32]) torch.Size([32])
tensor([ -5.1746,   2.3380,   2.3739,   4.0873,  -2.7718,  -5.6470, -12.6502,
         12.1036,  -2.0378,   4.8565], device='cuda:0')
tensor(7, device='cuda:0')
tensor(7, device='cuda:0')




torch.Size([32, 10]) torch.Size([32]) torch.Size([32])
tensor([-6.4334,  4.2927, -2.6381,  1.8912,  1.5940,  0.5524, -4.0676,  1.4546,
         1.9200,  2.1649], device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')




torch.Size([32, 10]) t

torch.Size([32, 10]) torch.Size([32]) torch.Size([32])
tensor([-7.7026,  8.8171, -0.4553,  1.3758,  1.0394, -2.0037, -2.2934,  1.8724,
         2.3842, -1.0676], device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')




torch.Size([32, 10]) torch.Size([32]) torch.Size([32])
tensor([-8.4340,  8.9664, -0.8700, -0.0483,  2.5153, -3.2711, -2.1037,  1.3106,
         4.0331, -0.5129], device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')




torch.Size([32, 10]) torch.Size([32]) torch.Size([32])
tensor([ -5.5854,   3.5592,   3.8988,   5.8706,  -4.5737,  -6.0941, -12.9523,
         13.8832,  -4.0488,   4.0980], device='cuda:0')
tensor(7, device='cuda:0')
tensor(7, device='cuda:0')




torch.Size([32, 10]) torch.Size([32]) torch.Size([32])
tensor([  3.5254,   1.3364,  15.9614,   7.5201, -17.5129,  -4.6587,  -4.7344,
         -0.3364,   5.5273,  -6.4750], device='cuda:0')
tensor(2, device='cuda:0')
tensor(2, device='cuda:0')




torch.Size([32, 10]) torch.Size(

tensor([-5.1598, -7.6575, -9.7725,  1.8233,  6.2245,  7.3433, -8.3555, -1.9283,
         2.7890, 12.2392], device='cuda:0')
tensor(9, device='cuda:0')
tensor(9, device='cuda:0')




torch.Size([32, 10]) torch.Size([32]) torch.Size([32])
tensor([ 2.3411, -2.6932,  1.1377, -1.1375,  0.4026,  5.6695, 10.2356, -9.4324,
         1.2249, -4.4084], device='cuda:0')
tensor(6, device='cuda:0')
tensor(6, device='cuda:0')




torch.Size([32, 10]) torch.Size([32]) torch.Size([32])
tensor([-2.6576, -0.0561,  0.5918,  7.3533, -7.0976,  2.8593, -8.8648,  0.3743,
         4.5704,  2.4166], device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')




torch.Size([32, 10]) torch.Size([32]) torch.Size([32])
tensor([ 4.0581, -6.0070,  5.5113, -4.6837,  3.1076, -0.3298, 10.7145, -8.6253,
         1.6791, -3.1366], device='cuda:0')
tensor(6, device='cuda:0')
tensor(6, device='cuda:0')




torch.Size([32, 10]) torch.Size([32]) torch.Size([32])
tensor([ 0.6390, -7.4383,  0.4021, -6.8489, 10.0331,

torch.Size([32, 10]) torch.Size([32]) torch.Size([32])
tensor([-5.3386,  7.1040,  1.5581,  2.0027, -1.8243, -1.5020, -2.2635,  2.0501,
         1.9466, -1.8819], device='cuda:0')
tensor(1, device='cuda:0')
tensor(1, device='cuda:0')




torch.Size([32, 10]) torch.Size([32]) torch.Size([32])
tensor([-0.0954, -6.6793, -1.7622, -3.1089,  8.5013, -0.3510, -0.2211, -0.4838,
        -2.3073,  4.8558], device='cuda:0')
tensor(4, device='cuda:0')
tensor(4, device='cuda:0')




torch.Size([32, 10]) torch.Size([32]) torch.Size([32])
tensor([-3.8753,  2.6085,  2.8473, 11.3620, -7.6028,  2.2147, -8.0801, -2.0179,
         3.0814,  0.8735], device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')




torch.Size([32, 10]) torch.Size([32]) torch.Size([32])
tensor([  1.6208,  -6.4099,  -0.6063,  -1.2247,  -3.8603,   6.4190,   1.8347,
        -11.1138,  12.2196,   1.0051], device='cuda:0')
tensor(8, device='cuda:0')
tensor(8, device='cuda:0')




torch.Size([32, 10]) torch.Size([32]) torc

torch.Size([32, 10]) torch.Size([32]) torch.Size([32])
tensor([-0.3141, -1.4669,  1.3631, 12.7483, -8.8639,  5.8449, -7.2057, -5.1919,
         2.7163,  1.1658], device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')




torch.Size([32, 10]) torch.Size([32]) torch.Size([32])
tensor([11.5588, -9.4725,  3.0234, -2.3544, -5.0930,  1.4055, -0.3630,  0.9816,
        -2.0775, -0.2796], device='cuda:0')
tensor(0, device='cuda:0')
tensor(0, device='cuda:0')




torch.Size([32, 10]) torch.Size([32]) torch.Size([32])
tensor([ -2.3101,  -3.1120,   3.4840,  14.7744, -11.5601,   2.6388, -15.0717,
         -0.2862,   4.8656,   5.2298], device='cuda:0')
tensor(3, device='cuda:0')
tensor(3, device='cuda:0')




torch.Size([32, 10]) torch.Size([32]) torch.Size([32])
tensor([ -3.9320,  -5.4804,  -6.0862,  -0.2056,   4.4190,  -0.1592, -11.3973,
          3.7708,   3.3821,  11.2179], device='cuda:0')
tensor(9, device='cuda:0')
tensor(9, device='cuda:0')




torch.Size([32, 10]) torch.Size(

torch.Size([32, 10]) torch.Size([32]) torch.Size([32])
tensor([ 2.8434, -4.1969, -1.8281,  4.0949, -3.9941,  8.3902,  0.1050, -4.8965,
         0.2685, -0.2511], device='cuda:0')
tensor(5, device='cuda:0')
tensor(5, device='cuda:0')




torch.Size([32, 10]) torch.Size([32]) torch.Size([32])
tensor([ 2.1481, -6.4543, -3.7076,  0.6929, -1.2405,  9.8797,  2.5691, -7.6209,
         2.9968,  1.3156], device='cuda:0')
tensor(5, device='cuda:0')
tensor(5, device='cuda:0')




torch.Size([32, 10]) torch.Size([32]) torch.Size([32])
tensor([ 1.3817, -5.9447, -6.0594,  2.4292, -3.4169, 14.4483,  1.1295, -7.7912,
         3.7751,  0.9768], device='cuda:0')
tensor(5, device='cuda:0')
tensor(5, device='cuda:0')




torch.Size([32, 10]) torch.Size([32]) torch.Size([32])
tensor([ 3.3994, -5.2027,  4.0072, -2.8918,  5.7823,  0.8159,  9.2865, -6.1266,
        -2.6371, -3.2246], device='cuda:0')
tensor(6, device='cuda:0')
tensor(6, device='cuda:0')




torch.Size([32, 10]) torch.Size([32]) torch.Size([32

torch.Size([32, 10]) torch.Size([32]) torch.Size([32])
tensor([ 1.7042, -2.5743,  3.3138, -4.9806,  2.4096,  2.0460, 11.7443, -7.3769,
         1.6437, -5.1125], device='cuda:0')
tensor(6, device='cuda:0')
tensor(6, device='cuda:0')




torch.Size([32, 10]) torch.Size([32]) torch.Size([32])
tensor([-4.2740, -3.1013, -4.0057, -4.4107, 11.8622, -0.7407, -1.2218,  0.1517,
         0.4115,  4.7629], device='cuda:0')
tensor(4, device='cuda:0')
tensor(4, device='cuda:0')




torch.Size([32, 10]) torch.Size([32]) torch.Size([32])
tensor([-2.6799, -7.3256, -1.8220,  0.8931,  3.9539, -1.4192, -8.5687,  1.7829,
         1.4346, 10.1714], device='cuda:0')
tensor(9, device='cuda:0')
tensor(9, device='cuda:0')




torch.Size([32, 10]) torch.Size([32]) torch.Size([32])
tensor([-1.1934, -8.7139, -2.2363, -1.3715,  4.9466, -1.5348, -7.2333,  2.6083,
         0.1845, 10.0447], device='cuda:0')
tensor(9, device='cuda:0')
tensor(9, device='cuda:0')




torch.Size([32, 10]) torch.Size([32]) torch.Size([32